# SIO - OWL - TD3 - OWLREADY2

## Préliminaires

### Importer les bibliothèques utiles

In [ ]:
from owlready2 import *
from SPARQLWrapper import *

# Change the following PATH if needed
#owlready2.JAVA_EXE='C:/Program Files/Common Files/Oracle/Java/javapath/java.exe'

### Hyperparamètres

In [ ]:
owlready2.JAVA_EXE='/usr/bin/java'

In [ ]:
# My ontology filename / préfixe pour les sauvegardes du peuplement
ONTO_NAME = "papiers_td3"
# Numéro identifiant la série de peuplement (pour éviter de supprimer les travux antérieurs
ONTO_PEUPLE_ID = "2"

## Importer notre ontologie

Ontologie "papiers" créée et modifiée avec Protégé

In [ ]:
onto = get_ontology(f"./{ONTO_NAME}.owl").load() # chargement d'une ontologie existante

## Analyse notre onthologie

In [ ]:
# Avons-nous des classes inconsistantes ?
print(list(onto.world.inconsistent_classes()))

In [ ]:
for i in onto.classes() : 
    print(i)    

In [ ]:
for i in onto.object_properties() : 
    print(i)    

In [ ]:
for i in onto.data_properties() : 
    print(i)    

In [ ]:
with onto:
    print(onto.Personne.iri)

## Ajouter des instances

### Définir quelques individus

In [ ]:
# Définition de quelques individus simple
# instance des classes Personne et Publication
with onto:
    balth = onto.Personne(name='Balthazar') 
    pub1 = onto.Publication(name='TD3')
    yolaine = onto.Personne(name='Yolaine')
    balth.aLu= [pub1]
    yolaine.aÉcrit = [pub1]

### Vérifier la création des instances

In [ ]:
with onto:
    print(balth.iri)
    print(yolaine.iri)
    print(pub1.iri)

### Tester la complétion des data properties

In [ ]:
with onto:
    balth.aPourInterets = ["big data, machine learning, knowledge graph"]
    balth.aPourMail = ["adresse@gmail.com"]
    balth.aPourNom = "M."
    balth.aPourPrenom = ["Balthazar, J-E"]
    
    yolaine.aPourInterets = ["OWL, web semantic, inférences"]
    yolaine.aPourMail = ["adresse@cs.fr"]
    yolaine.aPourNom = "B."
    yolaine.aPourPrenom = ["Yolaine"]
    
    pub1.aPourRésumé = ["Le TD n°3 du cours OWL - Owlready2"]
    pub1.aPourTitre = ["TD n°3"]
    pub1.aUnSousTitre = ["Owlready2"]

### Tester l'existance d'une personne dans notre ontologie

In [ ]:
# Préparer la requête
onto_subject = ONTO_NAME + ":Yolaine"
onto_object = ONTO_NAME + ":Personne"
req = "SELECT ?p WHERE {" + onto_subject + " ?p " + onto_object + ".}"

# Executer la requête sur l'ontologie
with onto:
    r = list(default_world.sparql(req, error_on_undefined_entities=False))

# Afficher le nombre de résultats (nombre de personnes avec ce même nom)
if len(r) == 0:
    print(f"{onto_subject} n'existe pas pour l'instant")
elif len(r) == 1 :
    print(f"{onto_subject} existe déjà")
else :
    print(f"{onto_subject} existe de trop nombreuses fois ! ")
    

### Nettoyer ses individus tests

In [ ]:
destroy_entity(balth)
destroy_entity(yolaine)
destroy_entity(pub1)

In [ ]:
# sauvegarde
onto.save("papiers_td3_peuple_" + ONTO_PEUPLE_ID + ".owl")

## A partir de HAL

## Peupler les auteurs et les publication du topic "Ingénierie assistée par ordinateur"

In [ ]:
# Le but ici est de récupérer toutes relations entre auteurs et publications
# Nous compléterons les data properties plus loin
sparql = SPARQLWrapper("http://sparql.archives-ouvertes.fr/sparql")
query = """
PREFIX hal:<http://data.archives-ouvertes.fr/schema/>
PREFIX dc: <http://purl.org/dc/terms/>

SELECT DISTINCT ?papier ?auteur
WHERE {
?papier hal:topic <https://data.archives-ouvertes.fr/subject/info.info-ai> .
?papier dc:creator ?node_auteur .
?node_auteur hal:person ?auteur .
}

"""
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
reponse = sparql.query().convert()

print("\tNombre d'individu :\n")
print(len(reponse['results']['bindings']))
print("\tLa liste des champs récupérés :\n")
print(reponse['head']['vars'])
print("\n\tUn aperçu de la liste de tous les individus :\n")
print(reponse['results']['bindings'][0:2])
print("\n\tUn individu = une ligne = une valeur de la liste :\n")
print(reponse['results']['bindings'][0])

### Peuplement automatique de quelques individus

In [ ]:
with onto:
    for ligne in reponse['results']['bindings']:
        
        # Récupérer l'id à la fin de l'IRI dans HAL
        author_id = ligne['auteur']['value'].split("/")[-1]
        paper_id = ligne['papier']['value'].split("/")[-1]
        
        # Créer les instances avec ces id  comme "name" dans notre ontologie
        auteur = onto.Personne(name=author_id)
        publication = onto.Publication(name=paper_id)
        
        # Ajouter les IRI de HAL dans les data properties de notre ontologie
        auteur.aPourIriExterne.append(ligne['auteur']['value'])
        publication.aPourIriExterne.append(ligne['papier']['value'])
        
        # Créer la relation entre un auteur et sa publication
        auteur.aÉcrit = [publication]

In [ ]:
# sauvegarde
onto.save("papiers_td3_peuple_" + ONTO_PEUPLE_ID + ".owl")

### Verifier le peuplement

In [ ]:
# Tester la relation "telle personne a écrit telle publication"
onto_subject = ONTO_NAME + ":" + reponse['results']['bindings'][0]['auteur']['value'].split("/")[-1]
onto_object = ONTO_NAME + ":" + reponse['results']['bindings'][0]['papier']['value'].split("/")[-1]

# Préparer la requête
req = "SELECT ?p WHERE { " + onto_subject  + " ?p " + onto_object + ".}"

with onto:
    r = list(default_world.sparql(req, error_on_undefined_entities=False))

print(f"{onto_subject} est lié < {len(r)} > fois à {onto_object}")
print("Relations existantes : ", *r)
    
    

## Peuplement de tous les auteurs des topics "Computational Engineering, Finance, and Science" et "Ingénierie assistée par ordinateur"

In [ ]:
# Le but ici est :
# de compléter les data properties des individus auteurs du topic info-ia déjà peuplés
# Par curiosité, de peupler l'ontologie avec des personnes qui ne sont pas forcément auteurs de publications info-ia
# En effet, nous n'avons peupler qu'avec des auteurs de inof-ia

sparql = SPARQLWrapper("http://sparql.archives-ouvertes.fr/sparql")
query = """
PREFIX hal:<http://data.archives-ouvertes.fr/schema/>
PREFIX dc: <http://purl.org/dc/terms/>

SELECT DISTINCT ?auteur ?aut_nom ?aut_prenom ?aut_mail ?aut_web
WHERE {

{ ?papier hal:topic <https://data.archives-ouvertes.fr/subject/info.info-ce> .}
UNION
{ ?papier hal:topic <https://data.archives-ouvertes.fr/subject/info.info-ai> .}
?papier dc:creator ?node_auteur .

?node_auteur hal:person ?auteur .
?auteur foaf:familyName ?aut_nom .
OPTIONAL {?auteur foaf:firstName ?aut_prenom}
OPTIONAL {?auteur foaf:mbox_sha1sum ?aut_mail}
OPTIONAL {?auteur foaf:homepage ?aut_web}
}
"""
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
reponse = sparql.query().convert()

print("\tNombre d'individus :\n")
print(len(reponse['results']['bindings']))
print("\tLa liste des champs récupérés :\n")
print(reponse['head']['vars'])
print("\n\tUn aperçu de la liste de tous les individus :\n")
print(reponse['results']['bindings'][0:2])
print("\n\tUn individu = une ligne = une valeur de la liste :\n")
print(reponse['results']['bindings'][0])

### Peuplement automatique de quelques individus

In [ ]:
with onto:
    for ligne in reponse['results']['bindings']:
        # Récupérer l'id à la fin de l'IRI dans HAL
        author_id = ligne['auteur']['value'].split("/")[-1]

        # Créer les instances avec ces id  comme "name" dans notre ontologie
        auteur = onto.Personne(name=author_id) 
        # Ajouter les IRI de HAL dans les data properties de notre ontologie
        auteur.aPourIriExterne.append(ligne['auteur']['value'])

        # Compléter les data properties de notre ontologie
        # S'assurer de leur existence pour éviter les Exceptions TypeError
        if 'aut_mail' in ligne :
            auteur.aPourMail = [ligne['aut_mail']['value']]

        if 'aut_nom' in ligne :
            auteur.aPourNom = ligne['aut_nom']['value']

        if 'aut_prenom' in ligne :
            auteur.aPourPrenom = [ligne['aut_prenom']['value']]

        if 'aut_web' in ligne :
            auteur.aPourSiteWeb.append(ligne['aut_web']['value'])
        

### Vérifier le peuplement des individus

In [ ]:
def isThereDuplicate(onto_name=ONTO_NAME, class_of_subject="Personne"):
    onto_object = ONTO_NAME + ":" + class_of_subject 
    # Récupérer le nombre d'interet par 
    req = "SELECT ?indiv WHERE { ?indiv a " + onto_object + ".}"
    
    with onto:
        reponse_base = list(default_world.sparql(req))

    # Récupérer le nombre d'individus distincts
    req = "SELECT DISTINCT ?indiv WHERE { ?indiv a " + onto_object + ".} ORDER BY ?indiv"
    with onto:
        reponse_distinct = list(default_world.sparql(req))
    
    nb_doublon = len(reponse_base) - len(reponse_distinct)
    if nb_doublon != 0 :
        return "Il existe des doublons de " + onto_object + ":\n > " + str(nb_doublon) + " doublons"

    return "Aucun doublon sur " + onto_object

In [ ]:
# Par manie, le but ici est de vérifier que nous n'avons pas injecté de doublon
print(isThereDuplicate(class_of_subject="Personne"))

In [ ]:
# sauvegarde
onto.save("papiers_td3_peuple_" + ONTO_PEUPLE_ID + ".owl")

## Peuplement de toutes les publications des topics "Informatique et théorie des jeux" et "Ingénierie assistée par ordinateur"

In [ ]:
# Le but ici est :
# de compléter les data properties des publications du topic info-ia déjà peuplé
# de peupler l'ontologie avec des publications qui n'ont pas forcément d'auteurs (par curiosité)
# En effet, dans notre premier peupement, seule les publications de info-ia ont un ou des auteurs identifiés

sparql = SPARQLWrapper("http://sparql.archives-ouvertes.fr/sparql")
query = """
PREFIX hal:<http://data.archives-ouvertes.fr/schema/>
PREFIX dc: <http://purl.org/dc/terms/>

SELECT DISTINCT ?individu ?pap_titre ?pap_sstitre ?pap_resume
WHERE {

{ ?individu hal:topic <https://data.archives-ouvertes.fr/subject/info.info-gt> .}
UNION
{ ?individu hal:topic <https://data.archives-ouvertes.fr/subject/info.info-ai> .}

?individu dc:title ?pap_titre .
OPTIONAL {?individu dc:abstract ?pap_resume}
OPTIONAL {?individu dc:alternative ?pap_sstitre}
FILTER (LANG(?pap_resume)="fr" && LANG(?pap_sstitre)="fr")
}
"""
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
reponse = sparql.query().convert()

print("\tNombre d'individus :\n")
print(len(reponse['results']['bindings']))
print("\tLa liste des champs récupérés :\n")
print(reponse['head']['vars'])
print("\n\tUn aperçu de la liste de tous les individus :\n")
print(reponse['results']['bindings'][0:2])
print("\n\tUn individu = une ligne = une valeur de la liste :\n")
print(reponse['results']['bindings'][0])


### Peuplement automatique des publications enrichies des data properties

In [ ]:
with onto:
    for ligne in reponse['results']['bindings']:
        # Récupérer l'id à la fin de l'IRI dans HAL
        individu_id = ligne['individu']['value'].split("/")[-1]
        
        # Créer les instances avec ces id  comme "name" dans notre ontologie
        individu = onto.Publication(name=individu_id) 
        # Ajouter les IRI de HAL dans les data properties de notre ontologie
        individu.aPourIriExterne.append(ligne['individu']['value'])
        
        # Compléter les data properties de notre ontologie
        # S'assurer de leur existence pour éviter les Exceptions TypeError
        if 'pap_titre' in ligne :
            individu.aPourTitre = [ligne['pap_titre']['value']]
            
        if 'pap_sstitre' in ligne :
            individu.aPourSousTitre = [ligne['pap_sstitre']['value']]
            
        if 'pap_resume' in ligne :
            individu.aPourSousTitre = [ligne['pap_resume']['value']]
            

### Vérifier le peuplement des publications

In [ ]:
# Par manie, le but ici est de vérifier que nous n'avons pas injecté de doublon
print(isThereDuplicate(class_of_subject="Publication"))

In [ ]:
# sauvegarde
onto.save("papiers_td3_peuple_" + ONTO_PEUPLE_ID + ".owl")

## Ajouter les intérêt aux personnes

In [ ]:
# Le but ici est :
# de compléter les data properties des personnes avec leurs intérêts

sparql = SPARQLWrapper("http://sparql.archives-ouvertes.fr/sparql")
query = """
PREFIX hal:<http://data.archives-ouvertes.fr/schema/>
PREFIX dc: <http://purl.org/dc/terms/>

SELECT DISTINCT ?individu ?indiv_interets
WHERE {

{ ?papier hal:topic <https://data.archives-ouvertes.fr/subject/info.info-ce> .}
UNION
{ ?papier hal:topic <https://data.archives-ouvertes.fr/subject/info.info-ai> .}
?papier dc:creator ?node_auteur .


?node_auteur hal:person ?individu .
OPTIONAL {?individu foaf:topic_interest ?indiv_interets}
FILTER (LANG(?indiv_interets)="fr")
}

ORDER BY ?individu
"""


sparql.setQuery(query)
sparql.setReturnFormat(JSON)
reponse = sparql.query().convert()

print("\tLa liste des champs récupérés :\n")
print(reponse['head']['vars'])
print("\n\tUn aperçu de la liste de tous les individus :\n")
print(reponse['results']['bindings'][0:10])
print("\n\tUn individu = une ligne = une valeur de la liste :\n")
print(reponse['results']['bindings'][0])


### Peuplement automatique des intérêts

In [ ]:
with onto:
    for ligne in reponse['results']['bindings']:
        # Récupérer l'id à la fin de l'IRI dans HAL
        individu_id = ligne['individu']['value'].split("/")[-1]
        
        # Créer les instances avec ces id  comme "name" dans notre ontologie
        individu = onto.Personne(name=individu_id) 
        # Ajouter les IRI de HAL dans les data properties de notre ontologie
        individu.aPourIriExterne.append(ligne['individu']['value'])
        
        # Compléter les data properties de notre ontologie
        # S'assurer de leur existence pour éviter les Exceptions TypeError
        if 'indiv_interets' in ligne :
            individu.aPourInterets = [ligne['indiv_interets']['value']]


In [ ]:
# Par manie, le but ici est de vérifier que nous n'avons pas injecté de doublon   
print(isThereDuplicate("Personne"))

In [ ]:
onto_object = ONTO_NAME + ":Personne" 
onto_predicat = ONTO_NAME + ":aPourInterets" 
# Récupérer le nombre d'interet par 
req = "SELECT ?auteur ?interet WHERE"
req += " { ?auteur a " + onto_object + ". ?auteur " + onto_predicat + " ?interet.}"
req += " ORDER BY ?auteur"

with onto:
    reponse_base = list(default_world.sparql(req))

# Récupérer le nombre d'individus distincts
req = "SELECT DISTINCT ?auteur ?interet WHERE "
req += "{ ?auteur a " + onto_object + ". ?auteur " + onto_predicat + " ?interet.}"
req += "ORDER BY ?auteur"
with onto:
    reponse_distinct = list(default_world.sparql(req))

nb_doublon = len(reponse_base) - len(reponse_distinct)
if nb_doublon != 0 :
    print("Il existe des doublons de " + onto_object + ":\n > " + str(nb_doublon) + " doublons")
else:
    print("Aucun doublon sur " + onto_object + ":aPourInterets")

In [ ]:
# sauvegarde
onto.save("papiers_td3_peuple_" + ONTO_PEUPLE_ID + ".owl")

## Détruire mon objet onto

In [ ]:
onto.destroy()